# data loading prototyping

This notebook is just a scratchpad for experimenting with how to load data from the ATL03 files

In [2]:
%cd ..

/mnt/c/Users/XCB/OneDrive - Van Oord/Documents/thesis/code


In [3]:
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import pandas as pd
from cftime import num2date, date2num, num2pydate
from datetime import datetime, timedelta
from atl_module.io.atl03_netcdf_loading import get_beams
import pdal

from atl_module.io.atl03_netcdf_loading import load_beam_array_ncds

In [4]:
h5filepath = "../data/test_sites/big_island/ATL03/processed_ATL03_20210301202807_10361007_005_01.nc"
beam = "gt3l"
# "C:\Users\maxli\OneDrive - Van Oord\Documents\thesis\data\ATL12_FL\239711552\processed_ATL12_20210613064553_12291101_005_02.nc"

In [5]:
ncds = Dataset(h5filepath)

In [6]:
delta_time_geophys = ncds.groups[beam].groups["geophys_corr"].variables["delta_time"][:]
delta_time_geoloc = ncds.groups[beam].groups["geolocation"].variables["delta_time"][:]

In [7]:
np.all(delta_time_geophys == delta_time_geoloc)

True

In [8]:
beamarray = ncds.groups[beam].groups["geolocation"].variables["segment_ph_cnt"][:]
# delta_time =

In [9]:
beamarray.filled()

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0, 781, 177,  89,   4,  84, 148, 100,  60,
        45, 115, 115,  50,  51,  89, 116,  58, 107,  51,  56, 130,  27,
       106,  72,  96,  49,  73, 118,  63,  59, 111,  57, 109,  82, 289,
       180, 179, 300, 177,  15, 329,  16,  99,  42, 140,  88,  57,  67,
        93,  69,  54,  99, 103,  37, 134, 103, 126,  83, 186, 185,  22,
       238,  41, 381,  47,  32, 187, 104,  11, 216,  65, 171, 126, 175,
        53, 128,  48, 127,  54, 108, 111, 149,  53, 102,  53, 169,  75,
        78,  60,  69,  96,  73,  67,  93,  82, 156,  52, 103,  55],
      dtype=int32)

In [10]:
df = load_beam_array_ncds(h5filepath, beam)

In [11]:
df

array([(-155.96004986, 19.86305704, -358.42383,        nan,       nan,         nan,        nan, '2021-03-01T20:29:54.269658', 0, 0,        nan,       nan,         nan,  0, nan),
       (-155.95985778, 19.86378328,  492.05646,        nan,       nan,         nan,        nan, '2021-03-01T20:29:54.269758', 0, 0,        nan,       nan,         nan,  0, nan),
       (-155.95986375, 19.86376054,  465.65384,        nan,       nan,         nan,        nan, '2021-03-01T20:29:54.269758', 0, 0,        nan,       nan,         nan,  0, nan),
       ...,
       (-155.96063612, 19.84549374,  119.74539,   99.97316, 19.687885, -0.26121876, 0.08435138, '2021-03-01T20:29:54.547658', 0, 0, 0.24075064, 1.4728825, -0.08638553, 55,  0.),
       (-155.96069214, 19.84527778, -130.78923, -150.56146, 19.687885, -0.26121876, 0.08435138, '2021-03-01T20:29:54.547658', 0, 0, 0.24075064, 1.4728825, -0.08638553, 55,  0.),
       (-155.96070489, 19.84522862, -187.81543, -207.58766, 19.687885, -0.26121876, 0.08435138, '2